In [0]:
%pip install --upgrade plotly


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
file_path = "/Volumes/workspace/default/housing_proj/Copy of Housing Bills 2023 to 2024 - Compiled Data.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)
df.head(5)

[Row(enteredby='Ahmed', 2023-2024='2023-2024', bill.no='AB-1490', org='Advisory Council for Sourcewise', support.lvl=3, sponsor=0),
 Row(enteredby='Ahmed', 2023-2024='2023-2024', bill.no='AB-1490', org='AIDS Healthcare Foundation', support.lvl=3, sponsor=1),
 Row(enteredby='Ahmed', 2023-2024='2023-2024', bill.no='AB-1490', org='California Apartment Association', support.lvl=3, sponsor=0),
 Row(enteredby='Ahmed', 2023-2024='2023-2024', bill.no='AB-1490', org='California Contract Cities Association', support.lvl=0, sponsor=0),
 Row(enteredby='Ahmed', 2023-2024='2023-2024', bill.no='AB-1490', org='City of Rosemead', support.lvl=0, sponsor=0)]

DELETE THE 'enterby' and 'year' COLUMN, WE DON'T NEED IT

In [0]:
file_path = "/Volumes/workspace/default/housing_proj/Copy of Housing Bills 2023 to 2024 - Compiled Data.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)
df = df.drop("enteredby",'2023-2024')

EDA

In [0]:
# Peek at the header that Spark actually read
df = (spark.read
          .csv("/Volumes/workspace/default/housing_proj/Copy of Housing Bills 2023 to 2024 - Compiled Data.csv",
               header=True, inferSchema=True))

df.printSchema()      # shows types *and* exact field names
# or simply:
print(df.columns)     # quick look at the raw strings

root
 |-- enteredby: string (nullable = true)
 |-- 2023-2024: string (nullable = true)
 |-- bill.no: string (nullable = true)
 |-- org: string (nullable = true)
 |-- support.lvl: integer (nullable = true)
 |-- sponsor: integer (nullable = true)

['enteredby', '2023-2024', 'bill.no', 'org', 'support.lvl', 'sponsor']


***VERY IMPORTANT TO MAKE SURE bill_col="'bill.no'" DO NOT LET PYSPARK TRY TO AUTOCORRECT

In [0]:
from pyspark.sql import functions as F

#UPDATE THESE TWO STRINGS IF NEEDED
bill_col = "`bill.no`"   # put back-ticks around names with dots/spaces
org_col  = "org"         # e.g. could be "org_id" or "organization"

df_norm = (df
           .withColumn("bill_id",
                       F.upper(F.trim(F.col(bill_col))))
           .withColumn("org_norm",
                       F.upper(F.trim(F.col(org_col)))))

distinct_bills = (df_norm
                  .select("bill_id")
                  .distinct()
                  .count())

distinct_orgs  = (df_norm
                  .select("org_norm")
                  .distinct()
                  .count())

print(f"Distinct bills: {distinct_bills}")
print(f"Distinct organisations: {distinct_orgs}")


Distinct bills: 178
Distinct organisations: 1676


In [0]:
#  SET-UP & CLEAN  (unchanged)
from pyspark.sql import functions as F
import plotly.express as px

path = "/Volumes/workspace/default/housing_proj/Copy of Housing Bills 2023 to 2024 - Compiled Data.csv"

df = (spark.read
          .option("header", True)
          .option("inferSchema", True)
          .csv(path))

df = (df
       .withColumnRenamed("support.lvl", "support_lvl")
       .withColumnRenamed("bill.no",     "bill_no"))

df_norm = (df
           .withColumn("bill_id",  F.upper(F.trim(F.col("bill_no"))))
           .withColumn("org_norm", F.upper(F.trim(F.col("org"))))
           .withColumn(
               "support_flag",
               F.when(F.col("support_lvl") == 3, "Support").otherwise("No Support")
           ))
#  AGGREGATE  (unchanged)
summary = (df_norm
    .groupBy("bill_id", "support_flag")
    .agg(
        F.countDistinct("org_norm").alias("org_count"),
        F.collect_set("org_norm").alias("org_list")
    ))

support_order = (summary
    .filter(F.col("support_flag") == "Support")
    .select("bill_id", F.col("org_count").alias("supporter_cnt")))

summary = summary.join(support_order, on="bill_id", how="left")

# ASCENDING BY SUPPORTERS
pdf = summary.toPandas()

# Rank by supporter count (fewest → most) so we can sort ascending later
pdf["rank_low_to_high"] = pdf["supporter_cnt"].fillna(-1).rank(method="min")

# Give SB-4 a rank *higher* than any other bill so it sinks to the bottom
max_rank = pdf["rank_low_to_high"].max()
pdf["plot_order"] = pdf.apply(
    lambda r: max_rank + 1 if r["bill_id"] == "SB-4" else r["rank_low_to_high"],
    axis=1
)

pdf = pdf.sort_values("plot_order")          # ascending order

# Hover label text & stance mapping
pdf["org_list_str"] = pdf["org_list"].apply(lambda lst: "<br>".join(sorted(lst)))
pdf["support_flag"] = pdf["support_flag"].map(
    {"Support": "✅ Support", "No Support": "🚫 No Support"}
)

#  PLOT
fig = px.bar(
    pdf,
    x="org_count",
    y="bill_id",
    color="support_flag",
    orientation="h",
    hover_data={
        "org_list_str": True,
        "org_count": ":,",
        "support_flag": False,
        "bill_id": False
    },
    labels={
        "org_count": "Number of Organizations",
        "bill_id":   "Bill ID",
        "org_list_str": "Organisations"
    },
    title="Housing Bills 2023-24",
    color_discrete_map={"✅ Support": "#2ca02c", "🚫 No Support": "#d62728"},
    template="plotly_white"
)

fig.update_layout(
    height=max(600, pdf["bill_id"].nunique() * 35),
    legend_title_text="Stance",
    hoverlabel_align="left",
    font=dict(size=14),
    bargap=0.25,
    yaxis=dict(showgrid=False),
    xaxis=dict(title=dict(standoff=10))
)

fig.show()
